In [1]:
import re
from nltk.corpus import stopwords
import pandas as pd
# 1x in je notebook draaien (als je dat nog niet had gedaan)
import nltk
nltk.download('stopwords')

df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")

# Algemene Nederlandse stopwoorden
DUTCH_STOPWORDS = set(stopwords.words("dutch"))

# Extra ruiswoorden die je NIET als match wilt zien

TEXT_STOPWORDS = DUTCH_STOPWORDS

def clean_text_for_matching(text: str) -> str:
    """Maak tekst schoon voor matching / uitleg."""
    text = str(text).lower()
    # haal alleen 'normale' woorden eruit
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    # filter stopwoorden en superkorte zooi
    tokens = [
        t for t in tokens
        if t not in TEXT_STOPWORDS and len(t) > 2
    ]
    return " ".join(tokens)


[nltk_data] Downloading package stopwords to C:\Users\Dhr.
[nltk_data]     Kootwijk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 📘 Feedback van de docent

**Hieronder staat gerichte feedback op jullie notebook, gebaseerd op de rubric en opdracht.**

**Wat gaat goed:**
- Jullie hebben een functionele content‑based recommender.
- De code is netjes opgebouwd en duidelijk leesbaar.
- Basis NLP‑cleaning en similarity werkt goed.

**Wat ontbreekt / moet beter voor de rubric:**
- **EDA mist volledig** (visualisaties, datakwaliteit, verkenning).
- **Datasetopschoning** niet duidelijk beschreven of onderbouwd.
- **Studentprofiel ontbreekt** → momenteel vergelijk je module met module.
- **Aanbeveling voor de gebruiker** ontbreekt (top 3–5 op basis van profiel).
- **Geen analyse van prestaties** (bijvoorbeeld uitleg waarom keuzes gemaakt zijn).
- **Geen optimalisatie/tuning** (bijv. parameters TF-IDF, embeddings testen).

**Aanbevolen verbeteringen:**
1. Voeg EDA toe (missing values, value counts, tekstlengtes, distributions).
2. Maak een *studentprofiel vector* (tags + interesses) en vergelijk daarmee.
3. Voeg uitleg toe bij elke stap: waarom deze keuze, hoe werkt het model?
4. Voeg experimenten toe met TF‑IDF parameters, of evt. embeddings.
5. Toon eindresultaten in een tabel + korte motivatie waarom deze modules passen.

Als jullie willen kan ik een aangepaste versie met inline comments maken. 


In [2]:
# Als je al combined_text hebt, gebruik die.
if "combined_text" in df.columns:
    base_text = df["combined_text"]
else:
    # Fallback: zelf combineren (pas aan naar wat jij wilt gebruiken)
    df["combined_text"] = (
        df["name"].fillna("") + " " +
        df["shortdescription"].fillna("") + " " +
        df["description"].fillna("") + " " +
        df["content"].fillna("") + " " +
        df["module_tags"].astype(str).fillna("") +" " +
        df["location"].astype(str).fillna("")
    )
    base_text = df["combined_text"]

# Schoongemaakte tekst voor recommender + uitleg
df["clean_text"] = base_text.apply(clean_text_for_matching)


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorizer trainen op de opgeschoonde tekst
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["clean_text"])

# Handig voor later bij match_terms
FEATURE_NAMES = vectorizer.get_feature_names_out()


In [4]:
import numpy as np

def extract_match_terms(student_vec, module_vec, max_terms: int = 8):
    """
    Zoek de overlappende woorden tussen studentprofiel en module.
    Geeft een lijst met duidelijke keywords terug.
    """
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])
    shared_idx = list(student_idx & module_idx)

    # Sorteer op naam (kan ook op frequentie, maar dit is prima voor nu)
    shared_idx = sorted(shared_idx)

    terms = [FEATURE_NAMES[i] for i in shared_idx]
    return terms[:max_terms]


In [5]:
def build_reason(match_terms):
    """Maak een korte NL zin waarom de module past."""
    if not match_terms:
        return "Deze module sluit aan bij je interesses op basis van de tekstuele overeenkomsten."
    if len(match_terms) == 1:
        return f"Deze module sluit aan bij je interesse in '{match_terms[0]}'."
    if len(match_terms) == 2:
        return f"Deze module sluit aan bij je interesses in '{match_terms[0]}' en '{match_terms[1]}'."
    # 3 of meer
    hoofd = ", ".join(match_terms[:-1])
    laatste = match_terms[-1]
    return f"Deze module sluit aan bij je interesses in {hoofd} en {laatste}."

def recommend_modules(student_profile: str, top_n: int = 5, studycredit: int = None, level: str = None, location: str = None) -> pd.DataFrame:
    filtered_df = df.copy()
    # Filteren op studiepunten en niveau
    if(studycredit is not None) :
        filtered_df = filtered_df[filtered_df['studycredit'] == studycredit].reset_index(drop=True)
    if(level is not None):
        filtered_df = filtered_df[filtered_df['level'].isin([level])].reset_index(drop=True)
    if(location is not None) :
        filtered_df = filtered_df[filtered_df['location'].str.contains(location, case=False, na=False)].reset_index(drop=True)

    X = vectorizer.fit_transform(filtered_df["clean_text"])
    
    """
    Geeft top N modules voor een studentprofiel,
    inclusief similarity, match_terms en een NL-uitleg.
    """
    
    # studenttekst schoonmaken
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])
    
    # cosine similarity
    sims = cosine_similarity(student_vec, X).flatten()
    # df_result = df.copy()
    filtered_df["similarity"] = sims

    # Top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    # match_terms + reden per rij
    match_terms_list = []
    reasons = []

    for idx in top.index:
        module_vec = X[idx]
        terms = extract_match_terms(student_vec, module_vec)
        match_terms_list.append(terms)
        reasons.append(build_reason(terms))

    top = top[["id","name", "shortdescription", "similarity", "location", "studycredit", "level"]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top


In [6]:
sam_profile = """
Ik ben derdejaarsstudent en ik heb veel interesse in zorginnovatie,
data en AI. Ik wil graag modules volgen over zorgtechnologie, 
innovatie in de zorg en creatieve manieren om met technologie impact te maken.
"""

# sam_studycredit = 30
# sam_level = "NLQF6"
# sam_location = "Breda"


sam_studycredit = None
sam_level = None
sam_location = None

pd.set_option("display.max_colwidth", None)

# aanbevelingen = recommend_modules(sam_profile, top_n=5, studycredit=sam_studycredit, level=sam_level, location=sam_location)
aanbevelingen = recommend_modules(sam_profile, top_n=5, )
display(aanbevelingen)


,id,name,shortdescription,similarity,location,studycredit,level,match_terms,reason
17,176,Technologie die ècht werkt: innovatie in zorg en welzijn,"Technologie, smart health, robotica, zorg, welzijn",0.295312,Breda,15,NLQF5,"[innovatie, technologie, zorg]","Deze module sluit aan bij je interesses in innovatie, technologie en zorg."
13,172,Zorg dichtbij,"Zorg op afstand, technologie, implementeren, zorg, veranderkunde",0.290159,Breda,15,NLQF5,"[technologie, zorg]",Deze module sluit aan bij je interesses in 'technologie' en 'zorg'.
140,331,Innovatiemanagement,"Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken. Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken.",0.209709,Tilburg,30,NLQF6,"[innovatie, technologie]",Deze module sluit aan bij je interesses in 'innovatie' en 'technologie'.
43,202,Palliatieve zorg,Palliatieve zorg Palliatieve zorg,0.203331,Breda,30,NLQF6,[zorg],Deze module sluit aan bij je interesse in 'zorg'.
72,255,Module 2.3 Data gestuurd Ontwerp (Data Driven Design),"Data driven design, big data, BIM/GIS, parametrisch ontwerpen, virtual reality, storytelling",0.198101,Den Bosch en Tilburg,15,NLQF5,"[data, maken]",Deze module sluit aan bij je interesses in 'data' en 'maken'.
